In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-fd377f76-7d9a-2cd2-d802-10c3b342bac2)


In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-02-12 12:15:48--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-02-12 12:15:48 (114 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [4]:
import helper_functions as hf

In [5]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-02-12 12:18:22--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.99.128, 172.253.117.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.99.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2023-02-12 12:18:22 (106 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [7]:
hf.unzip_data('nlp_getting_started.zip')

In [8]:
from matplotlib import pyplot as plt

In [9]:
import pandas as pd

In [86]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [87]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [88]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) # shuffle with random_state=42 for reproducibility
train_df_shuffled.head()


,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [89]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [90]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [91]:
len(train_df), len(test_df)

(7613, 3263)

In [92]:
import random

random_index = random.randint(0, len(train_df)-5)

In [93]:
random_index

4460

In [94]:
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f'Target: {target}','(real disaster)' if target > 0 else '(not real disaster)')
  print(f'Text:\n{text}\n')
  print('---\n')


Target: 0 (not real disaster)
Text:
I JUST SCREAMED SIDJSJDJEKDJSKDJD . I CANT STAND YOU ?? https://t.co/0Vcsafx9bY

---

Target: 0 (not real disaster)
Text:
Court back in session. Testimony continues with med. examiner discussing gunshot wounds #KerrickTrial

---

Target: 1 (real disaster)
Text:
#NJTurnpike å_ #NJTurnpike Reopens Hours After Truck Fire In? http://t.co/oABJZtbVyZ http://t.co/GPBXRrDc07

---

Target: 1 (real disaster)
Text:
The cryptic words that guided pilots on the Hiroshima bombing mission http://t.co/nSS5L64cvR #canada

---

Target: 1 (real disaster)
Text:
EU states squabble over immigration. UK-France Eurotunnel deluged with migrants. One dead as 'thousands storm' tunnel http://t.co/vf6CKLmCSX

---



In [95]:


from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42) # random state for reproducibility



In [96]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)


(6851, 6851, 762, 762)

In [97]:
# View the first 10 training sentences and their labels
train_sentences[:10], train_labels[:10]


(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

In [98]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# Note: in TensorFlow 2.6+, you no longer need "layers.experimental.preprocessing"
# you can use: "tf.keras.layers.TextVectorization", see https://github.com/tensorflow/tensorflow/releases/tag/v2.6.0 for more

# Use the default TextVectorization variables
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None


In [99]:
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))


15

In [100]:


# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)



In [101]:


# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)



In [102]:
# Create sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [103]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
Wildfire near Columbia River town is 50 percent contained http://t.co/gzGpWSqyMW #FireNews #WA http://t.co/ay49MTYyL8      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 146,  217, 3988,  499,  801,    9,  717, 1494, 3976,    1,    1,
        1739,    1,    0,    0]])>

In [104]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")


Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [105]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1") 

embedding

In [106]:


# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed



Original text:
I am a wreck      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.01105328,  0.01095992, -0.02260063, ...,  0.04179791,
         -0.04793797, -0.01557243],
        [-0.02097183,  0.01795337,  0.02347405, ..., -0.03852588,
         -0.02946994,  0.00725231],
        [-0.01686498, -0.04803884,  0.01681638, ...,  0.02104857,
         -0.02203274, -0.0453934 ],
        ...,
        [-0.0023914 ,  0.00369804, -0.03098704, ...,  0.00640032,
          0.02846298, -0.00614753],
        [-0.0023914 ,  0.00369804, -0.03098704, ...,  0.00640032,
          0.02846298, -0.00614753],
        [-0.0023914 ,  0.00369804, -0.03098704, ...,  0.00640032,
          0.02846298, -0.00614753]]], dtype=float32)>

## Model 0: Naive Bayes

In [107]:
     

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)





Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [108]:


baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")



Our baseline model achieves an accuracy of: 79.27%


In [109]:


# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]



array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [110]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [111]:


# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results



{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 1: Simmple dense model

In [112]:


# Create tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"



In [115]:


# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model



# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


# Fit the model
model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="simple_dense_model")])



Saving TensorBoard log files to: model_logs/simple_dense_model/20230212-161639
Epoch 1/5
215/215 [==============================] - 16s 48ms/step - loss: 0.6100 - accuracy: 0.6941 - val_loss: 0.5360 - val_accuracy: 0.7559
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.4414 - accuracy: 0.8190 - val_loss: 0.4692 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.3466 - accuracy: 0.8599 - val_loss: 0.4590 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2850 - accuracy: 0.8914 - val_loss: 0.4641 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2381 - accuracy: 0.9120 - val_loss: 0.4766 - val_accuracy: 0.7887


In [116]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 9ms/step - loss: 0.4766 - accuracy: 0.7887


[0.4766002595424652, 0.7887139320373535]

In [117]:
model_1_preds_probs = model_1.predict(val_sentences)
model_1_preds_probs.shape

24/24 [==============================] - 0s 5ms/step


(762, 1)

In [120]:
model_1_preds_probs = tf.squeeze(tf.round(model_1_preds_probs))

In [121]:
model_1_preds_probs[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [124]:
model_1_results = calculate_results(y_true = val_labels,
                                    y_pred = model_1_preds_probs)

model_1_results

{'accuracy': 78.87139107611549,
 'precision': 0.7926581572076621,
 'recall': 0.7887139107611548,
 'f1': 0.7860944810879305}

## Model 2: LSTM

In [139]:
tf.random.set_seed(42)

inputs = layers.Input(shape = (1,), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.LSTM(64, return_sequences = True)(x)
x = layers.LSTM(64)(x)
#x = layers.Dense(64, activation = 'relu')(x)
outputs = layers.Dense(1,activation = 'sigmoid')(x)

model_2 = tf.keras.Model(inputs, outputs, name = 'model_2_LSTM')

In [140]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_5 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm_7 (LSTM)               (None, 64)                49408     
                                                                 
 dense_10 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [142]:
model_2.compile(
    loss = 'binary_crossentropy',
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy']
)

In [143]:
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = [val_sentences, val_labels],
                              callbacks = [create_tensorboard_callback(SAVE_DIR, 
                                                                       'model_2_LSTM')])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230212-172812
Epoch 1/5
215/215 [==============================] - 12s 39ms/step - loss: 0.2214 - accuracy: 0.9229 - val_loss: 0.5380 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1614 - accuracy: 0.9413 - val_loss: 0.6066 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1270 - accuracy: 0.9527 - val_loss: 0.7423 - val_accuracy: 0.7822
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1062 - accuracy: 0.9610 - val_loss: 0.8454 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0822 - accuracy: 0.9656 - val_loss: 0.9422 - val_accuracy: 0.7835


In [144]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 7ms/step


array([[7.2045578e-03],
       [7.1649837e-01],
       [9.9960250e-01],
       [3.2050218e-02],
       [2.3753900e-04],
       [9.9840659e-01],
       [6.5601528e-01],
       [9.9978548e-01],
       [9.9965787e-01],
       [5.6057656e-01]], dtype=float32)

In [145]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [146]:
model_2_results = calculate_results(y_true = val_labels,
                                    y_pred = model_2_preds)

model_2_results

{'accuracy': 78.34645669291339,
 'precision': 0.7855568462558434,
 'recall': 0.7834645669291339,
 'f1': 0.7814394387142286}

## Model 3: GRU

In [156]:

# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers
model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_3")

# Build an RNN using the GRU cell
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_3_embedding(x)
# x = layers.GRU(64, return_sequences=True) # stacking recurrent cells requires return_sequences=True
x = layers.GRU(64)(x) 
# x = layers.Dense(64, activation="relu")(x) # optional dense layer after GRU cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")


In [157]:


# Compile GRU model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])



In [158]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_5 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru_7 (GRU)                 (None, 64)                37248     
                                                                 
 dense_19 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [159]:


# Fit model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "GRU")])



Saving TensorBoard log files to: model_logs/GRU/20230212-174335
Epoch 1/5
215/215 [==============================] - 11s 35ms/step - loss: 0.5212 - accuracy: 0.7341 - val_loss: 0.4552 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3164 - accuracy: 0.8714 - val_loss: 0.4928 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.2170 - accuracy: 0.9184 - val_loss: 0.5667 - val_accuracy: 0.7756
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1566 - accuracy: 0.9447 - val_loss: 0.6125 - val_accuracy: 0.7808
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1198 - accuracy: 0.9590 - val_loss: 0.6041 - val_accuracy: 0.7743


In [160]:


# Make predictions on the validation data
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs.shape, model_3_pred_probs[:10]



24/24 [==============================] - 1s 7ms/step


((762, 1), array([[0.33153138],
        [0.88687253],
        [0.9960938 ],
        [0.1376284 ],
        [0.01532403],
        [0.988289  ],
        [0.71711355],
        [0.9970573 ],
        [0.99661094],
        [0.44118357]], dtype=float32))

In [161]:
# Convert prediction probabilities to prediction classes
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [162]:


# Calcuate model_3 results
model_3_results = calculate_results(y_true=val_labels, 
                                    y_pred=model_3_preds)
model_3_results



{'accuracy': 77.42782152230971,
 'precision': 0.7743239391456158,
 'recall': 0.7742782152230971,
 'f1': 0.7732392305682255}

## Model 4: Bidirectional

In [165]:

# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)

# Build an RNN using the GRU cell
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64))(x) 
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")


In [166]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_5 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_21 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [167]:

model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])



In [168]:


# Fit model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "BIDIRECTIONAL")])


Saving TensorBoard log files to: model_logs/BIDIRECTIONAL/20230212-175248
Epoch 1/5
215/215 [==============================] - 15s 44ms/step - loss: 0.1326 - accuracy: 0.9580 - val_loss: 0.7511 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0780 - accuracy: 0.9701 - val_loss: 0.9594 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0597 - accuracy: 0.9737 - val_loss: 1.2177 - val_accuracy: 0.7756
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0541 - accuracy: 0.9772 - val_loss: 1.1908 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0454 - accuracy: 0.9790 - val_loss: 1.3512 - val_accuracy: 0.7664


In [170]:

# Make predictions on the validation data
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs.shape, model_4_pred_probs[:10]



24/24 [==============================] - 2s 4ms/step


((762, 1), array([[2.2238968e-03],
        [6.1419499e-01],
        [9.9997139e-01],
        [1.3995788e-01],
        [1.2452779e-05],
        [9.9984407e-01],
        [9.6020871e-01],
        [9.9998713e-01],
        [9.9997830e-01],
        [7.3475444e-01]], dtype=float32))

In [172]:
# Convert prediction probabilities to prediction classes
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [173]:


# Calcuate model_3 results
model_4_results = calculate_results(y_true=val_labels, 
                                    y_pred=model_4_preds)
model_4_results


{'accuracy': 76.64041994750657,
 'precision': 0.7679094748917975,
 'recall': 0.7664041994750657,
 'f1': 0.7642857598265804}

## Model 5: CNN

In [177]:


# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers
model_5_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_5")

# Create 1-dimensional convolutional layer to model sequences
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_5_embedding(x)
x = layers.Conv1D(filters=32, kernel_size=5, activation="relu")(x)
x = layers.GlobalMaxPool1D()(x)
# x = layers.Dense(64, activation="relu")(x) # optional dense layer
outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

# Compile Conv1D model
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Get a summary of our 1D convolution model
model_5.summary()



Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_5 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_5 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 conv1d (Conv1D)             (None, 11, 32)            20512     
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense_22 (Dense)            (None, 1)              

In [178]:


# Fit the model
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     "Conv1D")])



Saving TensorBoard log files to: model_logs/Conv1D/20230212-181003
Epoch 1/5
215/215 [==============================] - 14s 36ms/step - loss: 0.5675 - accuracy: 0.7152 - val_loss: 0.4713 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.3430 - accuracy: 0.8608 - val_loss: 0.4701 - val_accuracy: 0.7940
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2129 - accuracy: 0.9221 - val_loss: 0.5300 - val_accuracy: 0.7585
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1345 - accuracy: 0.9549 - val_loss: 0.6113 - val_accuracy: 0.7651
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0935 - accuracy: 0.9702 - val_loss: 0.6698 - val_accuracy: 0.7808


In [179]:


# Make predictions with model_5
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]



24/24 [==============================] - 0s 4ms/step


array([[0.22184305],
       [0.71498793],
       [0.9997261 ],
       [0.05716905],
       [0.00454145],
       [0.99049884],
       [0.910782  ],
       [0.9973901 ],
       [0.9987005 ],
       [0.20007418]], dtype=float32)

In [180]:


# Convert model_5 prediction probabilities to labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]



<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [181]:


# Calculate model_5 evaluation metrics 
model_5_results = calculate_results(y_true=val_labels, 
                                    y_pred=model_5_preds)
model_5_results



{'accuracy': 78.08398950131233,
 'precision': 0.7834583238190046,
 'recall': 0.7808398950131233,
 'f1': 0.778533312750939}

## Model 6: Transfer Learning

In [191]:
sample_sentence = random.choice(X_train)
sample_sentence

"I added a video to a @YouTube playlist http://t.co/1vjAlJA1SX GTA 5 Funny Moments - 'OBLITERATION!' (GTA 5 Online Funny Moments)"

In [195]:
import tensorflow_hub as hub
embed = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')
embeded_sample = embed([sample_sentence,
                        'When you can the universal sentece encoder on a sentence, it is ruined.'])


In [197]:

print(embeded_sample[1][:50])


tf.Tensor(
[ 0.03767121 -0.03241401  0.00408085  0.01597857  0.03960412 -0.05379619
 -0.06408953  0.04216724  0.06099323  0.0507115  -0.01161463 -0.05228469
  0.03538141  0.06870867  0.0644431  -0.10066213  0.00844224 -0.01198775
  0.03438548 -0.01264153  0.04575362  0.0173762   0.05059876  0.05105478
 -0.09299114  0.07794157  0.00671705  0.00348284  0.00580585  0.03431151
  0.03400987  0.01300427  0.02220811 -0.02784561 -0.10189427 -0.03729674
  0.07078869  0.07213486 -0.00352875 -0.01204607  0.00065451 -0.05643779
  0.04828857 -0.0280099  -0.01790225  0.00684707 -0.00861704 -0.03605994
 -0.05381995 -0.00665173], shape=(50,), dtype=float32)


In [198]:
sentence_encoder_layer = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4',
                                        input_shape = [],
                                        dtype = tf.string,
                                        trainable = False,
                                        name = 'USE')

In [202]:
# Create model using the Sequential API
model_6 = tf.keras.Sequential([
  sentence_encoder_layer, # take in sentences and then encode them into an embedding
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid")
], name="model_6_USE")

# Compile model
model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_24 (Dense)            (None, 64)                32832     
                                                                 
 dense_25 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [203]:


# Train a classifier on top of pretrained embeddings
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     "tf_hub_sentence_encoder")])



Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder/20230212-183206
Epoch 1/5
215/215 [==============================] - 7s 16ms/step - loss: 0.5069 - accuracy: 0.7843 - val_loss: 0.4474 - val_accuracy: 0.7992
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4150 - accuracy: 0.8146 - val_loss: 0.4361 - val_accuracy: 0.8110
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4001 - accuracy: 0.8224 - val_loss: 0.4318 - val_accuracy: 0.8110
Epoch 4/5
215/215 [==============================] - 5s 22ms/step - loss: 0.3927 - accuracy: 0.8273 - val_loss: 0.4272 - val_accuracy: 0.8136
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.3863 - accuracy: 0.8298 - val_loss: 0.4288 - val_accuracy: 0.8163


In [204]:


# Make predictions with USE TF Hub model
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]



24/24 [==============================] - 1s 18ms/step


array([[0.15214275],
       [0.7284311 ],
       [0.9879579 ],
       [0.19768031],
       [0.72325855],
       [0.664342  ],
       [0.98169726],
       [0.97915477],
       [0.92784834],
       [0.07971156]], dtype=float32)

In [205]:


# Convert prediction probabilities to labels
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]



<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [206]:


# Calculate model 6 performance metrics
model_6_results = calculate_results(val_labels, model_6_preds)
model_6_results



{'accuracy': 81.62729658792651,
 'precision': 0.8187546712946793,
 'recall': 0.8162729658792651,
 'f1': 0.8147089025083661}